<a href="https://colab.research.google.com/github/timidsurgeon/DS-Unit-1-Sprint-1-Dealing-With-Data/blob/master/module4-makefeatures/LS_DS_124_Make_features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

_Lambda School Data Science_

# Make features

Objectives
-  understand the purpose of feature engineering
-  work with strings in pandas
- work with dates and times in pandas

Links
- [Feature Engineering](https://en.wikipedia.org/wiki/Feature_engineering)
- Python Data Science Handbook
  - [Chapter 3.10](https://jakevdp.github.io/PythonDataScienceHandbook/03.10-working-with-strings.html), Vectorized String Operations
  - [Chapter 3.11](https://jakevdp.github.io/PythonDataScienceHandbook/03.11-working-with-time-series.html), Working with Time Series

# Warm-Up

Good evening DSPT2! Here's what we'll be doing for your warmup today.

https://alshams.github.io/responsibledata/bias-in-data-viz/

1. Read this :this: blog post
2. Go out on the internet and find a visualization that illustrates at least one of the biases mentioned in the article.
3. Drop a link to the visualization in Slack.
4. Show up at 6:30 Pacific in zoom and talk about your misleading visualization!

Presenting too much data
Sometimes, showing the big picture can make it hard to identify salient data or stories.

In Figure 2, the sheer number of lines makes it hard to focus on any one data point or trend. If the designer wanted to obscure some bad news, burying it in a massive amount of information could accomplish that—but it also makes the data visualization essentially worthless.

Figure 2: The number of lines in this data visualization makes it hard to isolate any one fact or trend

In other cases, the trends that appear when an entire data set is visualized are the opposite of trends that appear when subsets of that data are studied separately. This phenomenon, known as Simpson’s Paradox, is explained in Cathy O’Neil’s Weapons of Math Destruction using a national report on school performance as an example. The report, A Nation at Risk, which was the basis of wide-ranging public policy, stated that nationwide, high schoolers’ SAT scores had declined.

Examination of the data revealed that, while this was true in a big-picture sense, the period of data covered an era with tremendous growth in the number and range of students taking the exam; universities were admitting more minority and lower-income students, vastly increasing the numbers of these students taking the exam. When each cohort of students—analyzed according to income groups—was examined, the data actually showed increases in the average scores of each group.

When learners will need both a big-picture and a detailed visualization of data, the designer should consider creating a series of data visualizations. News media often do this with large data stories, showing a national map, for example with broad representations of data by region or state, then a series of more narrowly focused visualizations that highlight important trends, outliers, or other information.



## Website URL

https://learningsolutionsmag.com/articles/misleading-data-visualizations-can-confuse-deceive-learners

## Image URL

https://learningsolutionsmag.com/sites/default/files/inline-images/7441_Figure2.png

## Get LendingClub data

[Source](https://www.lendingclub.com/info/download-data.action)

In [0]:
!wget https://resources.lendingclub.com/LoanStats_2018Q4.csv.zip

In [0]:
!unzip LoanStats_2018Q4.csv.zip

In [0]:
!head LoanStats_2018Q4.csv

## Load LendingClub data

pandas documentation
- [`read_csv`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html)
- [`options.display`](https://pandas.pydata.org/pandas-docs/stable/options.html#available-options)

## Work with strings

For machine learning, we usually want to replace strings with numbers.

We can get info about which columns have a datatype of "object" (strings)

### Convert `int_rate`

Define a function to remove percent signs from strings and convert to floats

Apply the function to the `int_rate` column

### Clean `emp_title`

Look at top 20 titles

How often is `emp_title` null?

Clean the title and handle missing values

### Create `emp_title_manager`

pandas documentation: [`str.contains`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.str.contains.html)

## Work with dates

pandas documentation
- [to_datetime](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.to_datetime.html)
- [Time/Date Components](https://pandas.pydata.org/pandas-docs/stable/timeseries.html#time-date-components) "You can access these properties via the `.dt` accessor"

# ASSIGNMENT

- Replicate the lesson code.

- Convert the `term` column from string to integer.

- Make a column named `loan_status_is_great`. It should contain the integer 1 if `loan_status` is "Current" or "Fully Paid." Else it should contain the integer 0.

- Make `last_pymnt_d_month` and `last_pymnt_d_year` columns.

# STRETCH OPTIONS

You can do more with the LendingClub or Instacart datasets.

LendingClub options:
- There's one other column in the dataframe with percent signs. Remove them and convert to floats. You'll need to handle missing values.
- Modify the `emp_title` column to replace titles with 'Other' if the title is not in the top 20. 
- Take initiatve and work on your own ideas!

Instacart options:
- Read [Instacart Market Basket Analysis, Winner's Interview: 2nd place, Kazuki Onodera](http://blog.kaggle.com/2017/09/21/instacart-market-basket-analysis-winners-interview-2nd-place-kazuki-onodera/), especially the **Feature Engineering** section. (Can you choose one feature from his bulleted lists, and try to engineer it with pandas code?)
- Read and replicate parts of [Simple Exploration Notebook - Instacart](https://www.kaggle.com/sudalairajkumar/simple-exploration-notebook-instacart). (It's the Python Notebook with the most upvotes for this Kaggle competition.)
- Take initiative and work on your own ideas!

You can uncomment and run the cells below to re-download and extract the Instacart data

In [0]:
# !wget https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz

In [0]:
# !tar --gunzip --extract --verbose --file=instacart_online_grocery_shopping_2017_05_01.tar.gz

In [0]:
# %cd instacart_2017_05_01

# Link to lecture

https://youtu.be/Y39_6wZuv8A